In [29]:
import pandas as pd
import polars as pl

import numpy as np

In [ ]:
dict_dtypes = {'COD_HOSPITAL': dtype('int64'),
 'CIP_ENCRIPTADO': dtype('float64'),
 'SEXO': dtype('O'),
 'FECHA_NACIMIENTO': dtype('O'),
 'ETNIA': dtype('O'),
 'PROVINCIA': dtype('O'),
 'COMUNA': dtype('O'),
 'NACIONALIDAD': dtype('O'),
 'PREVISION': dtype('O'),
 'SERVICIO_SALUD': dtype('O'),
 'TIPO_PROCEDENCIA': dtype('O'),
 'TIPO_INGRESO': dtype('O'),
 'ESPECIALIDAD_MEDICA': dtype('O'),
 'TIPO_ACTIVIDAD': dtype('O'),
 'FECHA_INGRESO': dtype('O'),
 'SERVICIOINGRESO': dtype('O'),
 'FECHATRASLADO1': dtype('O'),
 'SERVICIOTRASLADO1': dtype('O'),
 'FECHATRASLADO2': dtype('O'),
 'SERVICIOTRASLADO2': dtype('O'),
 'FECHATRASLADO3': dtype('O'),
 'SERVICIOTRASLADO3': dtype('O'),
 'FECHATRASLADO4': dtype('O'),
 'SERVICIOTRASLADO4': dtype('O'),
 'FECHATRASLADO5': dtype('O'),
 'SERVICIOTRASLADO5': dtype('O'),
 'FECHATRASLADO6': dtype('O'),
 'SERVICIOTRASLADO6': dtype('O'),
 'FECHATRASLADO7': dtype('O'),
 'SERVICIOTRASLADO7': dtype('O'),
 'FECHATRASLADO8': dtype('O'),
 'SERVICIOTRASLADO8': dtype('O'),
 'FECHATRASLADO9': dtype('O'),
 'SERVICIOTRASLADO9': dtype('O'),
 'FECHAALTA': dtype('O'),
 'SERVICIOALTA': dtype('O'),
 'TIPOALTA': dtype('O'),
 'CONDICIONDEALTANEONATO1': dtype('float64'),
 'PESORN1': dtype('float64'),
 'SEXORN1': dtype('O'),
 'RN1ESTADO': dtype('float64'),
 'CONDICIONDEALTANEONATO2': dtype('float64'),
 'PESORN2': dtype('float64'),
 'SEXORN2': dtype('O'),
 'RN2ESTADO': dtype('float64'),
 'CONDICIONDEALTANEONATO3': dtype('float64'),
 'PESORN3': dtype('float64'),
 'SEXORN3': dtype('O'),
 'RN3ESTADO': dtype('float64'),
 'CONDICIONDEALTANEONATO4': dtype('float64'),
 'PESORN4': dtype('float64'),
 'SEXORN4': dtype('float64'),
 'RN4ESTADO': dtype('float64'),
 'DIAGNOSTICO1': dtype('O'),
 'DIAGNOSTICO2': dtype('O'),
 'DIAGNOSTICO3': dtype('O'),
 'DIAGNOSTICO4': dtype('O'),
 'DIAGNOSTICO5': dtype('O'),
 'DIAGNOSTICO6': dtype('O'),
 'DIAGNOSTICO7': dtype('O'),
 'DIAGNOSTICO8': dtype('O'),
 'DIAGNOSTICO9': dtype('O'),
 'DIAGNOSTICO10': dtype('O'),
 'DIAGNOSTICO11': dtype('O'),
 'DIAGNOSTICO12': dtype('O'),
 'DIAGNOSTICO13': dtype('O'),
 'DIAGNOSTICO14': dtype('O'),
 'DIAGNOSTICO15': dtype('O'),
 'DIAGNOSTICO16': dtype('O'),
 'DIAGNOSTICO17': dtype('O'),
 'DIAGNOSTICO18': dtype('O'),
 'DIAGNOSTICO19': dtype('O'),
 'DIAGNOSTICO20': dtype('O'),
 'DIAGNOSTICO21': dtype('O'),
 'DIAGNOSTICO22': dtype('O'),
 'DIAGNOSTICO23': dtype('O'),
 'DIAGNOSTICO24': dtype('O'),
 'DIAGNOSTICO25': dtype('O'),
 'DIAGNOSTICO26': dtype('O'),
 'DIAGNOSTICO27': dtype('O'),
 'DIAGNOSTICO28': dtype('O'),
 'DIAGNOSTICO29': dtype('O'),
 'DIAGNOSTICO30': dtype('O'),
 'DIAGNOSTICO31': dtype('O'),
 'DIAGNOSTICO32': dtype('O'),
 'DIAGNOSTICO33': dtype('O'),
 'DIAGNOSTICO34': dtype('O'),
 'DIAGNOSTICO35': dtype('O'),
 'PROCEDIMIENTO1': dtype('O'),
 'PROCEDIMIENTO2': dtype('O'),
 'PROCEDIMIENTO3': dtype('O'),
 'PROCEDIMIENTO4': dtype('float64'),
 'PROCEDIMIENTO5': dtype('O'),
 'PROCEDIMIENTO6': dtype('float64'),
 'PROCEDIMIENTO7': dtype('float64'),
 'PROCEDIMIENTO8': dtype('float64'),
 'PROCEDIMIENTO9': dtype('float64'),
 'PROCEDIMIENTO10': dtype('float64'),
 'PROCEDIMIENTO11': dtype('float64'),
 'PROCEDIMIENTO12': dtype('float64'),
 'PROCEDIMIENTO13': dtype('float64'),
 'PROCEDIMIENTO14': dtype('float64'),
 'PROCEDIMIENTO15': dtype('float64'),
 'PROCEDIMIENTO16': dtype('float64'),
 'PROCEDIMIENTO17': dtype('float64'),
 'PROCEDIMIENTO18': dtype('float64'),
 'PROCEDIMIENTO19': dtype('float64'),
 'PROCEDIMIENTO20': dtype('float64'),
 'PROCEDIMIENTO21': dtype('float64'),
 'PROCEDIMIENTO22': dtype('float64'),
 'PROCEDIMIENTO23': dtype('float64'),
 'PROCEDIMIENTO24': dtype('float64'),
 'PROCEDIMIENTO25': dtype('float64'),
 'PROCEDIMIENTO26': dtype('float64'),
 'PROCEDIMIENTO27': dtype('float64'),
 'PROCEDIMIENTO28': dtype('float64'),
 'PROCEDIMIENTO29': dtype('float64'),
 'PROCEDIMIENTO30': dtype('float64'),
 'MEDICOINTERV1_ENCRIPTADO': dtype('float64'),
 'FECHAPROCEDIMIENTO1': dtype('float64'),
 'FECHAINTERV1': dtype('O'),
 'ESPECIALIDADINTERVENCION': dtype('O'),
 'MEDICOALTA_ENCRIPTADO': dtype('int64'),
 'USOSPABELLON': dtype('float64'),
 'IR_29301_COD_GRD': dtype('int64'),
 'IR_29301_PESO': dtype('O'),
 'IR_29301_SEVERIDAD': dtype('int64'),
 'IR_29301_MORTALIDAD': dtype('int64'),
 'HOSPPROCEDENCIA': dtype('O')}

In [119]:
df = pd.read_csv('input/GRD_PUBLICO_2021.txt', sep='|')

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_8372\3830658916.py:1: DtypeWarning: Columns (26,27,28,29,30,31,32,33,47,83,84,85,86,87,88,89,90,92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('input/GRD_PUBLICO_2021.txt', sep='|')


In [120]:
df['IR_29301_PESO'] = df['IR_29301_PESO'].str.replace(',', '.').astype(float)
df['FECHAALTA'] = pd.to_datetime(df['FECHAALTA'])
df['FECHA_INGRESO'] = pd.to_datetime(df['FECHA_INGRESO'])

In [121]:
df['ESTANCIA'] = (df['FECHAALTA'] - df['FECHA_INGRESO'])

In [122]:
procedimientos = df.loc[:, 'PROCEDIMIENTO1': 'PROCEDIMIENTO30']
procedimientos['PROCEDIMIENTO1'] = procedimientos['PROCEDIMIENTO1'].replace('DESCONOCIDO', np.nan).astype(float)
procedimientos['PROCEDIMIENTO2'] = procedimientos['PROCEDIMIENTO2'].replace('DESCONOCIDO', np.nan).astype(float)
procedimientos['PROCEDIMIENTO3'] = procedimientos['PROCEDIMIENTO3'].replace('DESCONOCIDO', np.nan).astype(float)
procedimientos['PROCEDIMIENTO5'] = procedimientos['PROCEDIMIENTO5'].replace('DESCONOCIDO', np.nan).astype(float)

In [123]:
contiene_nuestros_proc = procedimientos.isin([31.1, 31.29]).sum(axis=1).astype(bool)
ambos_proc = contiene_nuestros_proc[contiene_nuestros_proc > 1].astype(bool)

In [124]:
proc_nacionales = df[contiene_nuestros_proc]
proc_torax = proc_nacionales.query('COD_HOSPITAL == 112103')

In [126]:
metricas = proc_nacionales.groupby('SERVICIO_SALUD').agg(n_egresos=('COD_HOSPITAL', 'count'),
                                              peso_medio=('IR_29301_PESO', 'mean'),
                                              estancia_media=('ESTANCIA', 'mean'))

In [127]:
metricas.to_excel('resumen_traqueostomia_2021.xlsx')
proc_nacionales.to_excel('crudo_traqueostomias_nacionales_2021.xlsx')